# UFRN Analytics

## Análise sobre os dados de Servidores da UFRN

O [Portal de Dados Abertos da UFRN](http://dados.ufrn.br) disponibiliza o recurso [Servidores](http://dados.ufrn.br/dataset/pessoas), uma base de dados com registro de todos os servidores ativos ou que se aposentaram pela instituição.  
  
Para cada servidor há informações de Nome, Matrícula SIAPE, Categoria, Cargo, Jornada de Trabalho, Regime Jurídico, Situação, Classe Funcional, Unidade de Lotação e Datas de Admissão e Desligamento.  

A Matrícula SIAPE, uma composição das colunas "siape" e "digito_siape", é um número de identificação para os servidores do executivo federal. 

A Categoria informa à qual das duas categorias existentes na instituição o servidor está vinculado: Docente ou Técnico Administrativo.  

O Cargo é uma indicação das atribuições do servidor, dentro da Categoria à qual está vinculado. Por exemplo: entre Docentes, há "Professor do Magistério Superior", "Professor de Ensino Básico, Técnico e Tecnológico etc; entre Técnicos Administrativos, existem "Assistentes em Administração", "Técnicos de Laboratório", "Engenheiros" etc.  

O Tipo de Jornada de Trabalho informa a carga horária semanal dedicada pelo servidor à instituição, ou, em alguns casos, se trata-se de Dedicação Exclusiva.

O Regime Jurídico de todos os servidores do quadro permanente da UFRN é Estatutário, portanto, essa coluna possui o mesmo valor em todas as linhas. 

A Situação do Servidor indica se ele está em atividade ou está aposentado.  

A Classe Funcional indica o estágio da carreira em que o servidor se encontra, de acordo com sua Categoria.  

A Unidade indica o local de lotação do servidor.

Por fim, as Datas de Admissão e Desligamento referem-se às datas de início e fim das atividades do servidores na instituição.

In [ ]:
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline

In [ ]:
servidores = pd.read_csv('servidores.csv', delimiter = ";")
servidores = servidores.drop(['Unnamed: 14'], axis=1)
print("Cinco primeiras linhas da base de dados 'Servidores':")
servidores.head()

In [ ]:
servidores['ano_admissao'] = pd.to_datetime(servidores['admissao']).dt.year
servidores['ano_desligamento'] = pd.to_datetime(servidores['data_desligamento']).dt.year
servidores['tempo_servico'] = servidores['ano_desligamento'] - servidores['ano_admissao']
servidores['count'] = 1

Diante dos dados que dispomos, uma análise preliminar pode ser feita em torno do número de admissões e aposentadorias registrados todos os anos. Esses dados nos permitirão plotar um perfil da curva do número de servidores ativos na UFRN ao longo dos anos e investigar momentos de expansão e retração do corpo docente e técnico administrativo e como tais momentos se relacionam com acontecimentos internos e externos à instituição. 

In [ ]:
docentes = servidores[servidores['categoria']=='Docente                       \n']

admissoes = docentes.groupby('ano_admissao').count()['count']

desligamentos = docentes.groupby('ano_desligamento').count()['count']

saldo = pd.DataFrame({'count_admissoes': admissoes, 'count_desligamentos': desligamentos})
saldo.loc[saldo['count_admissoes'].isnull(), 'count_admissoes'] = 0.0
saldo.loc[saldo['count_desligamentos'].isnull(), 'count_desligamentos'] = 0.0
saldo['count'] = saldo['count_admissoes'] - saldo['count_desligamentos']

ativos = pd.DataFrame({'ano': list(range(docentes['ano_admissao'].min(),2018))})
def check_year(year):
    year=int(year)
    count = docentes.loc[(docentes['ano_admissao']<=year) & ((docentes['ano_desligamento']>year) | (docentes['ano_desligamento'].isnull())), 'count'].count()
    return count
ativos['count'] = ativos['ano'].apply(check_year)

trace1 = go.Bar(
    name='Aposentadorias',
    x=desligamentos.index,
    y=desligamentos.values,
    textposition = 'auto',
    marker=dict(
        color='rgb(164,46,46)',
        ),
)

trace2 = go.Bar(
    name='Admissões',
    x=admissoes.index,
    y=admissoes.values,
    textposition = 'auto',
    marker=dict(
        color='rgb(142,212,95)',
        ),
)

trace3 = go.Scatter(
    name='Ativos',
    x=ativos['ano'],
    y=ativos['count'],
    textposition = 'auto',
    marker=dict(
        color='rgb(64,161,227)',
        ),
)
trace4 = go.Scatter(
    name='Saldo',
    x=saldo.index,
    y=saldo['count'],
    marker=dict(
        color='rgb(229,180,90)',
        ),
    mode = 'markers',
)

data = [trace1,trace2,trace3,trace4]
layout = go.Layout(
    title='Número de Docentes Ativos, Aposentadorias e Admissões por Ano na UFRN',
)

fig = go.Figure(data=data, layout=layout)
offline.init_notebook_mode()
offline.iplot(fig, filename='docentes_ativos_admissoes_desligamentos_ufrn')

In [ ]:
tecnicos = servidores[servidores['categoria']=='Técnico Administrativo']

admissoes = tecnicos.groupby('ano_admissao').count()['count']

desligamentos = tecnicos.groupby('ano_desligamento').count()['count']

saldo = pd.DataFrame({'count_admissoes': admissoes, 'count_desligamentos': desligamentos})
saldo.loc[saldo['count_admissoes'].isnull(), 'count_admissoes'] = 0.0
saldo.loc[saldo['count_desligamentos'].isnull(), 'count_desligamentos'] = 0.0
saldo['count'] = saldo['count_admissoes'] - saldo['count_desligamentos']

ativos = pd.DataFrame({'ano': list(range(tecnicos['ano_admissao'].min(),2018))})
def check_year(year):
    year=int(year)
    count = tecnicos.loc[(tecnicos['ano_admissao']<=year) & ((tecnicos['ano_desligamento']>year) | (tecnicos['ano_desligamento'].isnull())), 'count'].count()
    return count
ativos['count'] = ativos['ano'].apply(check_year)

trace1 = go.Bar(
    name='Aposentadorias',
    x=desligamentos.index,
    y=desligamentos.values,
    textposition = 'auto',
    marker=dict(
        color='rgb(164,46,46)',
        ),
)

trace2 = go.Bar(
    name='Admissões',
    x=admissoes.index,
    y=admissoes.values,
    textposition = 'auto',
    marker=dict(
        color='rgb(142,212,95)',
        ),
)

trace3 = go.Scatter(
    name='Ativos',
    x=ativos['ano'],
    y=ativos['count'],
    textposition = 'auto',
    marker=dict(
        color='rgb(64,161,227)',
        ),
)
trace4 = go.Scatter(
    name='Saldo',
    x=saldo.index,
    y=saldo['count'],
    marker=dict(
        color='rgb(229,180,90)',
        ),
    mode = 'markers',
)

data = [trace1,trace2,trace3,trace4]
layout = go.Layout(
    title='Número de Técnicos Administrativos Ativos, Aposentadorias e Admissões por Ano na UFRN',
)

fig = go.Figure(data=data, layout=layout)
offline.init_notebook_mode()
offline.iplot(fig, filename='tecnicos_ativos_admissoes_desligamentos_ufrn')

É necessário destacar alguns pontos a respeito de nossos dados.  
  
Só estão disponíveis registros de servidores em atividade ou que se aposentaram pela UFRN.
Servidores que se desligaram da insituição por outros motivos (redistribuição, exoneração etc) não foram registrados no dataset (nem mesmo sua admissão). 
O dataset também não nos fornece a data de nascimento dos servidores ou o motivo da aposentadoria. 
Apenas as datas de admissão e desligamento da UFRN estão disponíveis e isso nos permite saber o tempo de serviço na instituição. Todavia, um tempo de serviço pequeno pode ser suficiente para aposentar-se, a depender das normas vigentes à época, do tipo de aposentadoria e de eventuais averbações de tempo de serviço.

As curvas com os números de docentes e técnicos ativos ao longo dos anos possuem comportamentos semelhantes. 
Três momentos merecem destaque: o rápido crescimento do número de servidores ativos nas décadas de 70 e 80; a diminuição do número de servidores ativos na década de 90; o crescimento do número de servidores ativos desde 2007. Esses três momentos contam um pouco da história da instituição.

O Portal de Dados Abertos não disponibiliza um dataset com registros de alunos ativos ou ex-alunos. Comparar tais dados com os dados de servidores seria o ideal. Todavia, há o recurso [Cursos](http://dados.ufrn.br/dataset/cursos-ufrn/resource/949be3d1-e85b-4d0f-9f60-1d9a7484bb06), com registros dos cursos de todas as modalidades oferecidos pela UFRN.

In [ ]:
cursos = pd.read_csv('cursos-da-ufrn.csv', delimiter = ";")
print("Cinco primeiras linhas da base de dados 'Cursos':")
cursos.head()

Esse dataset possui uma coluna que informa a Data de Fundação de cada curso. O recurso contém diversos dados faltantes: dos 117 registros, 47 não têm uma Data de Fundação informada. Muitos cursos criados pela UFRN nos últimos anos não possuem essa coluna preenchida, mas a informação está disponível para cursos mais tradicionais. É possível identificar, por exemplo, que <b>entre 1970 e 1979 foram criados ao menos 28 novos cursos de graduação presenciais na UFRN </b>(18 bacharelados e 10 licenciaturas).  

Isso está diretamente relacionado com o primeiro momento que destacamos nas curvas, de rápido crescimento do número de servidores nos anos 1970 e 1980. Essa época possui altos índices de admissões e baixos índices de desligamentos. 1978 possui o maior número de admissões de docentes de toda a série; 1979, o de técnicos. O alto número de admissões se explica pela expansão da Universidade, que foi fundada em 1958, mas só nos anos 1970 e 1980 passou a ter a "cara" que conhecemos hoje. O baixo número de desligamentos se justifica pelo baixo número de servidores com condições de aposentadoria à época. O saldo entre admissões e desligamentos foi máximo nesse período e isso elevou o corpo de docentes e técnicos na UFRN, que na década de 80 já possuía um número de servidores ativos semelhante ao que tem hoje.

In [ ]:
cursos[(cursos['ano_fundacao']>=1970) & (cursos['ano_fundacao']<1980)]['grau_academico'].value_counts();

O segundo momento que destacamos é a diminuição do número de servidores ativos nos anos 1990. Trata-se de uma época de retração econômica no Brasil e a Universidade sentiu seus efeitos: não apenas interrompeu um ciclo de expansão como regrediu em diversas áreas.  

A década de 90 já apresenta registros de aposentadorias bem mais relevantes do que as décadas anteriores. 1991 e 1995 apresentam os maiores números de aposentadorias de docentes em toda a série e também estão entre os maiores quanto a aposentadoria de técnicos administrativos. Todavia, o número de admissões não esteve a altura no período. Não havendo compensação das perdas, o número de servidores ativos na instituição diminuiu.


Como identificar se os servidores se sentiram motivados a deixar a UFRN nesse período?
A partir dos dados que dispomos, também é possível analisar o tempo de serviço médio (ou mediano) dos servidores que se aposentaram em um dado ano.  

In [ ]:
servidores = servidores[servidores['tempo_servico']>0];

In [ ]:
trace1 = go.Scatter(
    name='Média',
    x=servidores.groupby('ano_desligamento').mean()['tempo_servico'].index,
    y=servidores.groupby('ano_desligamento').mean()['tempo_servico'].values,
    textposition = 'auto',
    marker=dict(
        color='rgb(64,161,227)',
        ),
)

trace2 = go.Scatter(
    name='Mediana',
    x=servidores.groupby('ano_desligamento').median()['tempo_servico'].index,
    y=servidores.groupby('ano_desligamento').median()['tempo_servico'].values,
    textposition = 'auto',
    marker=dict(
        color='rgb(222,90,13)',
        ),
)


data = [trace1,trace2]
layout = go.Layout(
    title='Tempo de Serviço (Anos) na UFRN dos Servidores Aposentados por Ano da Aposentadoria',
)

fig = go.Figure(data=data, layout=layout)
offline.init_notebook_mode()
offline.iplot(fig, filename='tempo_servico_aposentadoria_ufrn')

Uma observação importante sobre essa e as demais análises de tempo de serviço: foram excluídos registros (provavelmente errôneos) em que os anos de admissão e desligamento do servidor são os mesmos (às vezes, até a data é a mesma). É algo a ser corrigido na fonte dos datasets.

Nos anos 1990 a média do tempo de serviço entre aposentados pela UFRN é inferior ao que era na década de 80. Esse número de manteve por um período e desde os anos 2000 tem crescido continuamente. Hoje, é cerca de 15 anos mais alto do que há duas décadas. A década de 90 também tem o maior número de servidores aposentados com tempo de serviço inferior a 15 anos. É possível afirmar que os servidores estavam saindo da UFRN mais cedo neste período.

In [ ]:
trace1 = go.Scatter(
    name='Média',
    x=servidores[servidores['tempo_servico']<15].groupby('ano_desligamento').count()['count'].index,
    y=servidores[servidores['tempo_servico']<15].groupby('ano_desligamento').count()['count'].values,
    textposition = 'auto',
    marker=dict(
        color='rgb(64,161,227)',
        ),
)


data = [trace1]
layout = go.Layout(
    title='Número de Servidores da UFRN Aposentados com Tempo de Serviço Inferior a 15 Anos por Ano de Aposentadoria',
)

fig = go.Figure(data=data, layout=layout)
offline.init_notebook_mode()
offline.iplot(fig, filename='aposentados_15_tempo_ufrn')

O terceiro momento que destacamos refere-se à retomada do crescimento do número de servidores ativos na UFRN a partir de 2008.  

Em 2007 foi instituído o [Programa de Apoio a Planos de Reestruturação e Expansão das Universidades Federais - REUNI](http://www.planalto.gov.br/ccivil_03/_Ato2007-2010/2007/Decreto/D6096.htm). A partir de então, a UFRN, assim como outras universidades federais, recebeu recursos para compra de equipamentos, reforma e construção de prédios e, claro, contratação de pessoal. Isso explica o aumento do número de admissões desde então. O número de aposentadorias esteve estável na última década e foi geralmente superado pelo número de admissões, gerando um saldo positivo para o número de servidores ativos.  

2009 e 2010 são dois dos três anos com o maior número de novos docentes admitidos pela UFRN, dentre os que estão em atividade ou se aposentaram pela instituição. Em 2010 a Universidade finalmente supera o número de docentes ativos que teve durante a década de 1980.  

Em nossos gráficos, o crescimento do número de técnicos administrativos não acompanhou o crescimento do número de docentes. Uma explicação para isso é a maior rotatividade na primeira categoria (com maior frequência os servidores se desligam por exoneração para assumir outro vínculo) aliada ao fato de que nossos dados não contabilizam admissões de ex-servidores que não se aposentaram pela UFRN. Um servidor admitido em 2012 e que solicitou exoneração em 2015, por exemplo, não pôde ser computador como ativo entre 2012 e 2015 (o Portal de Dados Aberto da UFRN não nos fornece, hoje, recursos que nos possibilitem fazer isso).  

Vale salientar que a expansão da UFRN na última década foi justificada por novos compromissos assumidos pela instituição, respondendo uma demanda da sociedade. Ou seja, a contratação de docentes e técnicos se deu mediante a abertura de novos cursos e vagas de alunos nos níveis técnico, superior e de pós-graduação. Tendo em vista que a quantidade de aposentadorias tem se mantido estável, mas não é insignificante, um bom número de novas admissões será necessário todos os anos ao menos para que se mantenha a quantidade de servidores ativos na instituição e se possa atender aos compromissos acordados.

In [298]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
#docentes cargo
#tecnicos cargo

#tempo
#box plot tempo

In [299]:
IPython.load_extensions('usability/hide_input_all')

NameError: name 'IPython' is not defined